In [179]:
import pandas as pd
import numpy as np

In [200]:
df_train = pd.read_csv('../analysis/train.csv')
df_label_train = pd.read_csv('../analysis/train_label.csv')
df_sensor_log = pd.read_csv('../data/sensor_log.csv')

In [208]:
offset = np.max(df_sensor_log['block_id'].values) + 1
offset

19283

In [202]:
block_id_anomalous = df_label_train[df_label_train['anomalous'] == 1]['block_id'].values

In [203]:
df_anomalous = df_train[df_train['block_id'].isin(block_id_anomalous)]

In [204]:
df_desc = df_anomalous.describe()

In [205]:
df_desc.head()

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
count,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000
mean,8895.374046,2.447115,48.171005,52.139468,44.318828,630.680153,74.994393,13.799454,16.153960,15.449343,15.131636
std,5470.346861,0.211745,1.704151,1.505023,1.509306,38.456865,9.954861,0.999794,0.358564,0.381017,0.223266
min,197.000000,0.000000,40.104160,45.182290,40.190970,200.558441,0.000000,3.009259,14.712438,14.456552,14.147890
25%,4437.000000,2.450637,47.178820,51.176517,43.402775,629.629600,71.200263,13.418690,15.957750,15.183740,15.053530


In [206]:
mean = df_desc.loc['mean'].values[1:]
std = df_desc.loc['std'].values[1:]

In [227]:
data = []
for m, s in zip(mean, std):
    sample = np.random.normal(m, s, 10000).reshape(-1, 1)
    data.append(sample)
data = np.hstack(data)
data.shape

(10000, 10)

In [212]:
data = data.reshape(-1, 10, 10)
data.shape

(1000, 10, 10)

In [213]:
num_feats = df_sensor_log.columns.to_list()[2:]
df_feat = pd.DataFrame(columns=num_feats)

In [223]:
block_id = []
for i in range(data.shape[0]):
    block_id.append(offset + i)

In [224]:
data_block_id = np.array(block_id)[:, None]
data_block_id = np.repeat(data_block_id, 10, 1)
data_block_id = data_block_id.reshape(-1)

data_timestamp = ['2018-04-01 00:00:00'] * 10000

In [225]:
df_block_time = pd.DataFrame(data={
    'timestamp': data_timestamp,
    'block_id': data_block_id
})
df_block_time.head()

,timestamp,block_id
0,2018-04-01 00:00:00,19283
1,2018-04-01 00:00:00,19283
2,2018-04-01 00:00:00,19283
3,2018-04-01 00:00:00,19283
4,2018-04-01 00:00:00,19283


In [228]:
df_data = pd.DataFrame(columns=num_feats, data=data)
df_data.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2.443477,48.680898,50.504485,45.452339,618.457305,56.571440,13.709435,15.835443,15.559798,15.142161
1,2.263859,49.228748,51.437323,43.456435,649.474994,80.262261,13.990597,16.631511,15.673469,14.873253
2,2.600869,46.965823,53.121143,43.172117,611.938201,87.063616,13.489349,16.441751,15.270024,15.327701
3,2.458276,47.100368,51.946741,42.236402,626.461171,72.586921,12.357139,16.316415,15.687014,14.757318
4,2.323883,47.830747,50.832909,44.339837,576.113791,74.208547,13.807148,16.250277,14.888644,15.080641


In [229]:
df_all = pd.concat([df_block_time, df_data], axis=1)
df_all.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-01 00:00:00,19283,2.443477,48.680898,50.504485,45.452339,618.457305,56.571440,13.709435,15.835443,15.559798,15.142161
1,2018-04-01 00:00:00,19283,2.263859,49.228748,51.437323,43.456435,649.474994,80.262261,13.990597,16.631511,15.673469,14.873253
2,2018-04-01 00:00:00,19283,2.600869,46.965823,53.121143,43.172117,611.938201,87.063616,13.489349,16.441751,15.270024,15.327701
3,2018-04-01 00:00:00,19283,2.458276,47.100368,51.946741,42.236402,626.461171,72.586921,12.357139,16.316415,15.687014,14.757318
4,2018-04-01 00:00:00,19283,2.323883,47.830747,50.832909,44.339837,576.113791,74.208547,13.807148,16.250277,14.888644,15.080641


In [230]:
df_label = pd.DataFrame(columns=df_label_train.columns.to_list())

In [232]:
df_label['block_id'] = block_id
df_label['anomalous'] = [1] * 1000

In [234]:
df_all.to_csv('../analysis/new_data/new_data.csv', index=False)
df_label.to_csv('../analysis/new_data/df_new_data_label.csv', index=False)